In [1]:
%matplotlib inline
""" handling files support packages """
from glob import glob

""" logic support packages """
import numpy as np
import pandas as pd

""" plot support packages """
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from IPython.display import Image
#import pydotplus

""" image trasformation packages """
from PIL import Image
import skimage.io as skio

""" statistical data visualization packages"""
import seaborn as sns

""" machine learning functions """
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.externals import joblib
from sklearn.model_selection import GridSearchCV

""" seaborn configurations """
sns.set_style('white')
sns.set_context('talk')
plt.rcParams['figure.figsize'] = 20, 10

In [2]:
#from funcoes import ler_letras

In [3]:
save = True

In [4]:
#letras, letters_dict = ler_letras("../letras.csv")

In [5]:
data = pd.read_csv("../letras.csv", header = 0, sep=",")
list(data.columns.values)

['path', 'rotulo', 'caixa_alta_baixa']

In [6]:
print("Temos {0} classes: {1}".format(len(list(set(data.rotulo))), sorted(list(set(data.rotulo)))))

Temos 35 classes: ['1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [7]:
img_pixels = Image.open(data['path'][0])

In [8]:
#for i, file in enumerate(list(data['path'])):
    #print(file)

In [9]:
X = []
y = []

for i, file in enumerate(list(data['path'])):
    label = data['rotulo'][i]
    img_pixels = list(Image.open(file).getdata())
    X.append(img_pixels)
    y.append(label)

In [10]:
#from sklearn import preprocessing

#le = preprocessing.LabelEncoder()
#le.fit(y)
#ley = le.transform(y)

# Unsupervised Learnind: Dimensionality Reduction

In [11]:
print("temos {0} imagens cada uma com {1} dimensoes".format(len(X), len(X[0])))

temos 1140 imagens cada uma com 1750 dimensoes


In [12]:
#from sklearn.manifold import Isomap

In [13]:
#iso = Isomap(n_components = 2)
#data_projected = iso.fit_transform(X)

In [14]:
#data_projected.shape

In [15]:
#plt.scatter(data_projected[:, 0], data_projected[:, 1], c=ley,
#            edgecolor='none', alpha=0.5, cmap=plt.cm.get_cmap('nipy_spectral'))
#plt.colorbar(label='digita label', ticks=range(10))
#plt.clim(-0.5, 9.5)

A base relativa aos pixels esta muito confusa, as classes estao muito misturadas para os features (imagens).

# Divisao da base em treino e teste e aplicacao de algoritmos de aprendizado

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [17]:
%%time
from sklearn.neural_network import MLPClassifier

MLPparams = {'hidden_layer_sizes':(100, 20, 50),
             'activation' : ('identity', 'logistic', 'tanh', 'relu'),
             'solver' : ('lbfgs', 'sgd', 'adam'),
             'learning_rate': ('constant', 'invscaling', 'adaptative')
            }

svr = MLPClassifier(max_iter = 200000)
clfMLP = GridSearchCV(svr, MLPparams)
clfMLP = MLPClassifier()
clfMLP.fit( X = X_train, y = y_train )
print( "Score: {0}".format(clfMLP.score(X_test, y_test)) )
print( "Best parameters: " + str(clfMLP.get_params) )

Score: 0.023391812865497075
Best parameters: <bound method BaseEstimator.get_params of MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)>
CPU times: user 41.7 s, sys: 33.6 s, total: 1min 15s
Wall time: 26 s


In [25]:
%%time
from sklearn.ensemble import RandomForestClassifier
clfRF_balanced = RandomForestClassifier()


RFparams = {'n_estimators' : (1000, 500, 200, 100),
           'criterion' : ('gini', 'entropy'),
           'min_samples_split' : (2, 10, 40),
           'min_samples_leaf' : (1, 5, 10, 40)}

svr = RandomForestClassifier()
clfRF_balanced = GridSearchCV(svr, RFparams)
clfRF_balanced.fit( X = X_train, y = y_train )
print( "Score: {0}".format(clfRF_balanced.score(X_test, y_test)) )
print( "Best parameters: " + str(clfRF_balanced.best_params_) )

Score: 0.6549707602339181
Best parameters: {'n_estimators': 500, 'criterion': 'entropy', 'min_samples_split': 2, 'min_samples_leaf': 1}
CPU times: user 32min 55s, sys: 31.3 s, total: 33min 27s
Wall time: 33min 36s


In [26]:
%%time
from sklearn.svm import SVC
clf = SVC() # kernel = 'poly', degree = 2, max_iter = 100000

SVMparams = [#{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
             #        'C': [1, 10, 100, 1000]},
            #{'kernel': ['linear'], 'C': [1, 10, 100, 1000]},
            {'kernel' : ['poly'], 'degree': [1, 2, 3],
                     'C': [1, 10, 100, 1000]}]

svr = SVC()
clf = GridSearchCV(svr, SVMparams)

clf.fit(X_train, y_train)
print( "Score: {0}".format(clf.score(X_test, y_test)) )
print( "Best parameters: " + str(clf.best_params_) )

Score: 0.564327485380117
Best parameters: {'kernel': 'poly', 'C': 1, 'degree': 2}
CPU times: user 3min 42s, sys: 1.04 s, total: 3min 43s
Wall time: 3min 45s


In [ ]:
#import tensorflow as tf

#image_size = 35 * 50
#n_classes = len(list(set(data.rotulo)))
#x = tf.placeholder(tf.float32, [None, image_size])
#W = tf.Variable(tf.zeros([image_size, n_classes]))
#b = tf.Variable(tf.zeros([n_classes]))

In [ ]:
#imgs = skio.imread_collection(list(data['path']))
#imgA = crop_char(imgs[0], 0)

In [ ]:
#print(clf.predict([item for sublist in imgA.tolist() for item in sublist]))

In [29]:
%%time
if save:
    from sklearn.ensemble import RandomForestClassifier
    clf = RandomForestClassifier(n_estimators = 500, criterion = 'entropy',
                                  min_samples_split = 2, min_samples_leaf= 1)

    clf.fit(X_train, y_train)
    print( "Score: {0}".format(clf.score(X_test, y_test)) )

Score: 0.6432748538011696
CPU times: user 18.6 s, sys: 108 ms, total: 18.7 s
Wall time: 18.7 s


In [30]:
if save: joblib.dump(clf, 'classifier_image.pkl') 